In [1]:
!pip install kagglehub

In [4]:
import kagglehub
import pandas as pd
from pathlib import Path
import os, sys, re

# Download latest version
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\baong\.cache\kagglehub\datasets\jp797498e\twitter-entity-sentiment-analysis\versions\2


#### Train Data

In [5]:
file = [f for f in os.listdir(Path(f'{path}').absolute()) if f.endswith('training.csv')]
file

['twitter_training.csv']

In [6]:
sys.setrecursionlimit(5000)
script_location = Path('social-network/NLP-Sentiment-Analyzer').absolute()

# Expanding the dispay of text sms column
pd.set_option('display.max_colwidth', 1)

csv_folder = [f for f in os.listdir(Path(f'{path}').absolute()) if f.endswith('training.csv')]
data = []
for f in csv_folder:
    data.append(pd.read_csv(path + '/' + f, header=None))

df = pd.concat(data, ignore_index=True)
df.rename(columns={0:'Tweet_ID', 1: 'Entity', 2: 'Sentiment', 3: 'Tweet_content'}, inplace=True)
df['Sentiment'] = df['Sentiment'].replace('Negative'  , -1)
df['Sentiment'] = df['Sentiment'].replace('Irrelevant',  0)
df['Sentiment'] = df['Sentiment'].replace('Neutral'   ,  0)
df['Sentiment'] = df['Sentiment'].replace('Positive'  ,  1)
df['Tweet_content'] = df['Tweet_content'].astype(str).fillna('')
df.shape, df.columns


C:\Users\baong\AppData\Local\Temp\ipykernel_1656\1227249578.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sentiment'] = df['Sentiment'].replace('Positive'  ,  1)


((74682, 4),
 Index(['Tweet_ID', 'Entity', 'Sentiment', 'Tweet_content'], dtype='object'))

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet_ID       74682 non-null  int64 
 1   Entity         74682 non-null  object
 2   Sentiment      74682 non-null  int64 
 3   Tweet_content  74682 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [8]:
df.head(10)

,Tweet_ID,Entity,Sentiment,Tweet_content
0,2401,Borderlands,1,"im getting on borderlands and i will murder you all ,"
1,2401,Borderlands,1,"I am coming to the borders and I will kill you all,"
2,2401,Borderlands,1,"im getting on borderlands and i will kill you all,"
3,2401,Borderlands,1,"im coming on borderlands and i will murder you all,"
4,2401,Borderlands,1,"im getting on borderlands 2 and i will murder you me all,"
5,2401,Borderlands,1,"im getting into borderlands and i can murder you all,"
6,2402,Borderlands,1,So I spent a few hours making something for fun. . . If you don't know I am a HUGE @Borderlands fan and Maya is one of my favorite characters. So I decided to make myself a wallpaper for my PC. . Here is the original image versus the creation I made :) Enjoy! pic.twitter.com/mLsI5wf9Jg
7,2402,Borderlands,1,"So I spent a couple of hours doing something for fun... If you don't know that I'm a huge @ Borderlands fan and Maya is one of my favorite characters, I decided to make a wallpaper for my PC.. Here's the original picture compared to the creation I made:) Have fun! pic.twitter.com / mLsI5wf9Jg"
8,2402,Borderlands,1,So I spent a few hours doing something for fun... If you don't know I'm a HUGE @ Borderlands fan and Maya is one of my favorite characters.
9,2402,Borderlands,1,So I spent a few hours making something for fun. . . If you don't know I am a HUGE RhandlerR fan and Maya is one of my favorite characters. So I decided to make myself a wallpaper for my PC. . Here is the original image versus the creation I made :) Enjoy! pic.twitter.com/mLsI5wf9Jg


In [9]:
import wordcloud as wordcloud
from collections import Counter
import streamlit as st
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pickle
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [116]:
!pip install scikit-learn

  Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl.metadata (15 kB)
Using cached scikit_learn-1.6.0-cp312-cp312-win_amd64.whl (11.1 MB)


In [10]:
def preprocessor(text):
    """ Return a cleaned version of text
        
    """   
    # Remove HTML markup
    text = re.sub(r'<[^>]*>', '', text)
    # Replace n't at note
    text = re.sub(r"n't", 'not', text)
    # Remove emoticons
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', '', text)
    # Remove any non-word character and digit
    text = re.sub(r'[^A-Za-z ]+', '', text)
    # Also Convert to lower case
    text = (re.sub(r'[\W]+', ' ', text.lower()))    
    return text


def tokenizer_porter(text):
    """Split a text into list of words and apply stemming technic
    
    """
    porter = PorterStemmer()
    return [porter.stem(word) for word in text.split()]
          
def convert(sentiment):
    """Convert from 5 sentiment to 3 sentiment
    
    """
    if sentiment < 0:
        sentiment = -1 # Negative
    if sentiment == 0:
        sentiment = 0 # Neutral
    if sentiment > 0:
        sentiment = 1 # Positive
    
    return sentiment

In [11]:
# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baong\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# Convert
df['converted_sentiment'] = df['Sentiment'].apply(convert)
df['preprocessed'] = df['Tweet_content'].apply(preprocessor)
df['keyword_list'] = df['preprocessed'].apply(tokenizer_porter)
pickle.dump(df, open(f'DataFrame.sav', 'wb')) ## Save Dataframe to .sav file

In [17]:
df.head(5)

,Tweet_ID,Entity,Sentiment,Tweet_content,converted_sentiment,preprocessed,keyword_list
0,2401,Borderlands,1,"im getting on borderlands and i will murder you all ,",1,im getting on borderlands and i will murder you all,"[im, get, on, borderland, and, i, will, murder, you, all]"
1,2401,Borderlands,1,"I am coming to the borders and I will kill you all,",1,i am coming to the borders and i will kill you all,"[i, am, come, to, the, border, and, i, will, kill, you, all]"
2,2401,Borderlands,1,"im getting on borderlands and i will kill you all,",1,im getting on borderlands and i will kill you all,"[im, get, on, borderland, and, i, will, kill, you, all]"
3,2401,Borderlands,1,"im coming on borderlands and i will murder you all,",1,im coming on borderlands and i will murder you all,"[im, come, on, borderland, and, i, will, murder, you, all]"
4,2401,Borderlands,1,"im getting on borderlands 2 and i will murder you me all,",1,im getting on borderlands and i will murder you me all,"[im, get, on, borderland, and, i, will, murder, you, me, all]"


In [18]:
# Train model on non-converted sentiment
stopword = stopwords.words('english')
X = df['Tweet_content']
y = df['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# tfid = TfidfVectorizer(stop_words=stop_words, tokenizer=tokenizer_porter, preprocessor=preprocessor)
count = CountVectorizer(stop_words=stopword, preprocessor=preprocessor)

# Construct pipeline
clf = Pipeline([('vect', count), ('clf', LogisticRegression(random_state=0, max_iter=1000, n_jobs=6))])
clf.fit(X, y)
prediction = clf.predict(X_test)

#Print Result
print('accuracy:',accuracy_score(y_test, prediction))
print('confusion matrix:\n',confusion_matrix(y_test, prediction))
print('classification report:\n',classification_report(y_test, prediction))
    
# Save trained model to disk
pickle.dump(clf, open('model1.sav', 'wb'))

f:\Work\GitHub\social-network\NLP-Sentiment-Analyzer\NLP-2412122\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arenot', 'couldnot', 'didnot', 'doesnot', 'donot', 'hadnot', 'hasnot', 'havenot', 'isnot', 'mightnot', 'mustnot', 'neednot', 'shanot', 'shes', 'shouldnot', 'shouldve', 'thatll', 'wasnot', 'werenot', 'wonot', 'wouldnot', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


accuracy: 0.9148423378188392
confusion matrix:
 [[4005  336  128]
 [ 176 5887  189]
 [ 124  319 3773]]
classification report:
               precision    recall  f1-score   support

          -1       0.93      0.90      0.91      4469
           0       0.90      0.94      0.92      6252
           1       0.92      0.89      0.91      4216

    accuracy                           0.91     14937
   macro avg       0.92      0.91      0.91     14937
weighted avg       0.92      0.91      0.91     14937



In [19]:
# Train model on converted sentiment
stopword = stopwords.words('english')
X = df['Tweet_content']
y2 = df['converted_sentiment']
X_train, X_test, y_train_2, y_test_2 = train_test_split(X, y2, test_size=0.2, random_state=0)
# tfid = TfidfVectorizer(stop_words=stop_words, tokenizer=tokenizer_porter, preprocessor=preprocessor)
count = CountVectorizer(stop_words=stopword, preprocessor=preprocessor)

# Construct pipeline
clf = Pipeline([('vect', count), ('clf', LogisticRegression(random_state=0, max_iter=1000, n_jobs=6))])
clf.fit(X, y2)
prediction2 = clf.predict(X_test)

#Print Result
print('accuracy:',accuracy_score(y_test_2,prediction2))
print('confusion matrix:\n',confusion_matrix(y_test_2,prediction2))
print('classification report:\n',classification_report(y_test_2,prediction2))
    
# Save trained model to disk
pickle.dump(clf, open('model2.sav', 'wb'))

f:\Work\GitHub\social-network\NLP-Sentiment-Analyzer\NLP-2412122\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arenot', 'couldnot', 'didnot', 'doesnot', 'donot', 'hadnot', 'hasnot', 'havenot', 'isnot', 'mightnot', 'mustnot', 'neednot', 'shanot', 'shes', 'shouldnot', 'shouldve', 'thatll', 'wasnot', 'werenot', 'wonot', 'wouldnot', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(


accuracy: 0.9148423378188392
confusion matrix:
 [[4005  336  128]
 [ 176 5887  189]
 [ 124  319 3773]]
classification report:
               precision    recall  f1-score   support

          -1       0.93      0.90      0.91      4469
           0       0.90      0.94      0.92      6252
           1       0.92      0.89      0.91      4216

    accuracy                           0.91     14937
   macro avg       0.92      0.91      0.91     14937
weighted avg       0.92      0.91      0.91     14937



### Test

In [45]:
test = [f for f in os.listdir(Path(f'{path}').absolute()) if f.endswith('validation.csv')]
test

['twitter_validation.csv']

In [21]:
sys.setrecursionlimit(5000)
script_location = Path('social-network/NLP-Sentiment-Analyzer').absolute()

# Expanding the dispay of text sms column
pd.set_option('display.max_colwidth', 1)

csv_folder_test = [f for f in os.listdir(Path(f'{path}').absolute()) if f.endswith('validation.csv')]
data_test = []
for f in csv_folder_test:
    data_test.append(pd.read_csv(path + '/' + f, header=None))

df1 = pd.concat(data_test, ignore_index=True)
df1.rename(columns={0:'Tweet_ID', 1: 'Entity', 2: 'Sentiment', 3: 'Tweet_content'}, inplace=True)
df1['Sentiment'] = df1['Sentiment'].replace('Negative'  , -1)
df1['Sentiment'] = df1['Sentiment'].replace('Irrelevant',  0)
df1['Sentiment'] = df1['Sentiment'].replace('Neutral'   ,  0)
df1['Sentiment'] = df1['Sentiment'].replace('Positive'  ,  1)
df1['Tweet_content'] = df1['Tweet_content'].astype(str).fillna('')
df1.shape, df1.columns


C:\Users\baong\AppData\Local\Temp\ipykernel_1656\527537233.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Sentiment'] = df1['Sentiment'].replace('Positive'  ,  1)


((1000, 4),
 Index(['Tweet_ID', 'Entity', 'Sentiment', 'Tweet_content'], dtype='object'))

In [22]:
df1

,Tweet_ID,Entity,Sentiment,Tweet_content
0,3364,Facebook,0,"I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣"
1,352,Amazon,0,BBC News - Amazon boss Jeff Bezos rejects claims company acted like a 'drug dealer' bbc.co.uk/news/av/busine…
2,8312,Microsoft,-1,@Microsoft Why do I pay for WORD when it functions so poorly on my @SamsungUS Chromebook? 🙄
3,4371,CS-GO,-1,"CSGO matchmaking is so full of closet hacking, it's a truly awful game."
4,4433,Google,0,Now the President is slapping Americans in the face that he really did commit an unlawful act after his acquittal! From Discover on Google vanityfair.com/news/2020/02/t…
...,...,...,...,...
995,4891,GrandTheftAuto(GTA),0,"⭐️ Toronto is the arts and culture capital of Canada, it’s no wonder! If you want to start planning, be sure to check out our GTA Real Estate market report for Fall 2020, it has all the info you need to finally make a move! blog.remax.ca/toronto-housin… twitter.com/kevinyoufool/s…"
996,4359,CS-GO,0,tHIS IS ACTUALLY A GOOD MOVE TOT BRING MORE VIEWERS.\n\nI was one of those people who got hooked into csgo by watching tournaments first before playing the game. And seeing these players grew is like a netflix docu series for me. Can't wait for 2021.
997,2652,Borderlands,1,Today sucked so it’s time to drink wine n play borderlands until the sun comes up so I can hate myself all day tomorrow.
998,8069,Microsoft,1,Bought a fraction of Microsoft today. Small wins.


In [44]:
df1['Sentiment'][df1['Sentiment']==1].count() / df1.shape[0] * 100

np.float64(27.700000000000003)

In [37]:
# -*- coding: utf-8 -*-
import pandas as pd
import re
import pickle
from pathlib import Path

script_location = Path('social-network/NLP-Sentiment-Analyzer').absolute()

def load_model():
    model1 = pickle.load(open('model1.sav', 'rb'))
    return model1

def predict(phrase):
    return int(model1.predict([phrase]))

def result(df_test):
    result = []
    for i in range(df_test.shape[0]):
        if df_test['Sentiment'].iloc[i] == df_test['Sentiment_Test'].iloc[i]:
            result.append('T')
        else:
            result.append('F')
    df_test['Result'] = result
    return df_test

model1 = load_model()
def main():  
    csv_folder_test = [f for f in os.listdir(Path(f'{path}').absolute()) if f.endswith('validation.csv')]
    data_test = []
    for f in csv_folder_test:
        data_test.append(pd.read_csv(path + '/' + f, header=None))

    df_test = pd.concat(data_test, ignore_index=True)
    df_test.rename(columns={0:'Tweet_ID', 1: 'Entity', 2: 'Sentiment', 3: 'Tweet_content'}, inplace=True)
    df_test['Sentiment'] = df_test['Sentiment'].replace('Negative'  , -1)
    df_test['Sentiment'] = df_test['Sentiment'].replace('Irrelevant',  0)
    df_test['Sentiment'] = df_test['Sentiment'].replace('Neutral'   ,  0)
    df_test['Sentiment'] = df_test['Sentiment'].replace('Positive'  ,  1)
    df_test['Tweet_content'] = df_test['Tweet_content'].astype(str).fillna('')
    df_test['Sentiment_Test'] = df_test['Tweet_content'].apply(predict)
    try:
        result(df_test)
        df_test.to_csv('final_result.csv')
        print('Export Successfully!!')
    except:
        print('Fail to export')
        raise
if __name__ == '__main__':
    main()


C:\Users\baong\AppData\Local\Temp\ipykernel_1656\1065487642.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_test['Sentiment'] = df_test['Sentiment'].replace('Positive'  ,  1)
f:\Work\GitHub\social-network\NLP-Sentiment-Analyzer\NLP-2412122\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['arenot', 'couldnot', 'didnot', 'doesnot', 'donot', 'hadnot', 'hasnot', 'havenot', 'isnot', 'mightnot', 'mustnot', 'neednot', 'shanot', 'shes', 'shouldnot', 'shouldve', 'thatll', 'wasnot', 'werenot', 'wonot', 'wouldnot', 'youd', 'youll', 'youre', 'youve'] not in stop_words.
  warnings.warn(
C:\Users\baong\AppData\Local\Temp\ipykernel_1656\106

Export Successfully!!
